<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Разделение-на-train,-test" data-toc-modified-id="Разделение-на-train,-test-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение на train, test</a></span></li><li><span><a href="#Fit-MyDecisionTree" data-toc-modified-id="Fit-MyDecisionTree-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fit MyDecisionTree</a></span></li><li><span><a href="#Predict-MyDecisionTree" data-toc-modified-id="Predict-MyDecisionTree-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Predict MyDecisionTree</a></span></li><li><span><a href="#Train-with-sklearn" data-toc-modified-id="Train-with-sklearn-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Train with sklearn</a></span></li><li><span><a href="#Сравнение-метрик" data-toc-modified-id="Сравнение-метрик-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Сравнение метрик</a></span></li></ul></div>

# Задание  

1. Напишите свой алгоритм построения дерева решений для задачи бинарной классификации.  
    - критерий информативности - Энтропия Шеннона
    - критерии останова - максимальная глубина, кол-во элементов в листе, прирост энтропии < x
2. Сравните результат работы своего алгоритма с sklearn    

3. (дополнительно)  Попробуйте не делать One-Hot-Encoding для категориальных переменных, а добавить их обработку в свой алгоритм. Сравните качество работы алгоритма с предыдущим решением.

## Подготовка данных
Рассмотрим задачу "Титаник" https://www.kaggle.com/c/titanic/data. Необходимо предсказать выживет пассажир или нет.

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix 

In [2]:
# считаем данные из файла в pandas DataFrame
df = pd.read_csv("train.csv")

# зафиксируем целевую переменную и удалим ее из данных
y = df['Survived']
df.drop('Survived', axis=1, inplace=True)

In [3]:
# удалим признаки PassengerId, Name, Ticket и Cabin из данных
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# заполним пропуски в признаке Age обучающей выборки медианным значением
df['Age'].fillna(df['Age'].median(), inplace=True)

#заполним пропуски в признаке Embarked обучающей выборки самыми частыми значениями этого признака
df['Embarked'].fillna(df['Embarked'].value_counts().idxmax(), inplace=True)

#заменим категориальные признаки, используя One-Hot-Encoding
categorical = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
df = pd.concat([df, pd.get_dummies(df[categorical], columns=categorical, drop_first=True)],axis=1)

df.drop(categorical, axis=1, inplace=True)

In [4]:
df.head()

,Age,Fare,Pclass_2,Pclass_3,Sex_male,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_Q,Embarked_S
0,22.0,7.2500,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,38.0,71.2833,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,26.0,7.9250,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,35.0,53.1000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,35.0,8.0500,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## Разделение на train, test

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size = 0.1, random_state = 13)

In [6]:
print ('Train: ' + str(len(X_train)))
print ('Test: ' + str(len(X_test)))

Train: 801
Test: 90


## Fit MyDecisionTree  
Напишите свою функцию построения дерева.

Структуру дерева можно задать, например, словарем следующего вида:  
```python
{'feature_name': 'Age', # название фичи 
'threshold': 20, # порог разбиения
'left': 0,  # ссылка на левое поддерево, задан доминирующий класс
'right': 1} # ссылка на правое поддерево, задан доминирующий класс
```

Для представления дерева создал класс `MyDecisionTreeClassifier`

In [7]:
# Класс узла
class Node:
    def __init__(self, index, threshold, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.threshold = threshold  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле


# Класс терминального узла (листа)
class Leaf:
    def __init__(self, data, labels):
        self.data = data  # значения признаков
        self.labels = labels  # y_true
        self.prediction = self.predict()  # y_pred

    def predict(self):
        classes = {}  # "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его
        prediction = max(classes, key=classes.get)
        return prediction


# Само дерево
class MyDecisionTreeClassifier:
    def __init__(self, max_depth, min_samples_leaf, criterion='entropy'):
        self.depth = max_depth
        self.criterion = eval(f'self.{criterion}')
        self.min_samples_leaf = min_samples_leaf
        self.model = None

    def fit(self, X, y):
        data = X.to_numpy()
        target = y.to_numpy()
        self.model = self.build_tree(data, target, 1)

    def predict(self, data):
        data = data.to_numpy()
        classes = []
        for obj in data:
            prediction = self.classify_object(obj, self.model)
            classes.append(prediction)
        return classes

    def quality(self, left_targets, right_targets, criterion):
        p = float(left_targets.shape[0]) / (left_targets.shape[0] + right_targets.shape[0])
        return criterion - p * self.criterion(left_targets) - (1 - p) * self.criterion(
            right_targets)

    def entropy(self, target):
        def p(val):
            return val / target.size

        class_counter = np.unique(target, return_counts=True)[1]
        return -(np.array([p(val) * np.log2(p(val)) for val in class_counter])).sum()

    def build_tree(self, data, target, level):

        if level >= self.depth:
            return Leaf(data, target)
        level += 1

        quality, t, index = self.find_best_split(data, target)

        if quality == 0:
            return Leaf(data, target)  # считаем прогноз для листьев

        true_data, false_data, true_labels, false_labels = self.split(data, target, index, t)

        true_branch = self.build_tree(true_data, true_labels, level)
        false_branch = self.build_tree(false_data, false_labels, level)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return Node(index, t, true_branch, false_branch)

    def split(self, data, target, index, threshold):
        left = np.where(data[:, index] <= threshold)
        right = np.where(data[:, index] > threshold)

        true_data = data[left]
        false_data = data[right]
        true_labels = target[left]
        false_labels = target[right]

        return true_data, false_data, true_labels, false_labels

    def find_best_split(self, data, target):
        criterion = self.criterion(target)

        best_quality = 0
        best_threshold = None
        best_index = None

        n_features = data.shape[1]  # кол-во признаков

        for index in range(n_features):
            threshold_values = [row[index] for row in data]  # берем столбец/признак с соотв. индексом

            for threshold in threshold_values:  # проход по признаку
                true_data, false_data, true_labels, false_labels = self.split(data, target, index,
                                                                              threshold)

                if len(true_data) < self.min_samples_leaf or len(false_data) < self.min_samples_leaf:
                    continue

                current_quality = self.quality(true_labels, false_labels, criterion)

                if current_quality > best_quality:
                    best_quality, best_threshold, best_index = current_quality, threshold, index

        return best_quality, best_threshold, best_index

    def classify_object(self, obj, node):
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.threshold:
            return self.classify_object(obj, node.true_branch)
        else:
            return self.classify_object(obj, node.false_branch)



## Train with sklearn 

Обучите дерево, используя библиотеку sklearn. Задайте те же параметры, что и при обучении своего дерева.  

Сравните метрики и попробуйте улучшить ваше дерево.

In [8]:
from sklearn.tree import DecisionTreeClassifier
from termcolor import colored

## Сравнение метрик

In [9]:
print('depth \t my roc train \t sklearn roc train \t my roc test \t sklearn roc test')

my_roc_train_results = []
my_roc_test_results = []

for depth in range(1, 10):
    clf_tree = DecisionTreeClassifier(max_depth=depth, min_samples_leaf=5, criterion='entropy', random_state=0)
    clf_tree.fit(X_train, y_train)
    y_pred_test = clf_tree.predict(X_test)
    y_pred_train = clf_tree.predict(X_train)
    
    
    my_tree = MyDecisionTreeClassifier(max_depth=depth, min_samples_leaf=5, criterion='entropy')
    my_tree.fit(X_train, y_train)
    my_pred_test = my_tree.predict(X_test)
    my_pred_train = my_tree.predict(X_train)
    
    
    # метрики при обучении, используя библиотеку sklearn
    skl_roc_train = roc_auc_score(y_train, y_pred_train)
    skl_roc_test = roc_auc_score(y_test, y_pred_test)

    # метрики при обучении, используя собственный алгоритм
    my_roc_train = roc_auc_score(y_train, my_pred_train)
    my_roc_test = roc_auc_score(y_test, my_pred_test)

    my_roc_train_results.append(my_roc_train)
    my_roc_test_results.append(my_roc_test)

    
    color = 'green' if my_roc_train >= skl_roc_train else 'red'
    print(f'{depth}', colored(f'{my_roc_train:18.4f} {skl_roc_train:18.4f}', color), end='')
    
    color = 'green' if my_roc_test >= skl_roc_test else 'red'
    print(colored(f'{my_roc_test:18.4f} {skl_roc_test:18.4f}', color))

depth 	 my roc train 	 sklearn roc train 	 my roc test 	 sklearn roc test
1             0.5000             0.7625            0.5000             0.8108
2             0.7625             0.7625            0.8108             0.8108
3             0.7625             0.8055            0.8108             0.8524
4             0.8055             0.8020            0.8524             0.7895
5             0.8020             0.8090            0.7895             0.7810
6             0.8090             0.8222            0.7810             0.7726
7             0.8245             0.8256            0.7641             0.7726
8             0.8279             0.8387            0.7641             0.8609
9             0.8411             0.8387            0.8362             0.8609


# Без One-Hot-Encoding для категориальных переменных

In [10]:
# считаем данные из файла в pandas DataFrame
df = pd.read_csv("train.csv")

# зафиксируем целевую переменную и удалим ее из данных
y = df['Survived']
df.drop('Survived', axis=1, inplace=True)

# удалим признаки PassengerId, Name, Ticket и Cabin из данных
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# заполним пропуски в признаке Age обучающей выборки медианным значением
df['Age'].fillna(df['Age'].median(), inplace=True)

#заполним пропуски в признаке Embarked обучающей выборки самыми частыми значениями этого признака
df['Embarked'].fillna(df['Embarked'].value_counts().idxmax(), inplace=True)

df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [11]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size = 0.1, random_state = 13)

In [12]:
print('depth \t new roc train \t old roc train \t new roc test \t old roc test')

for depth in range(1, 10):

    
    my_tree = MyDecisionTreeClassifier(max_depth=depth, min_samples_leaf=5, criterion='entropy')
    my_tree.fit(X_train, y_train)
    my_pred_test = my_tree.predict(X_test)
    my_pred_train = my_tree.predict(X_train)
    

    # метрики при обучении, используя собственный алгоритм
    my_roc_train = roc_auc_score(y_train, my_pred_train)
    my_roc_test = roc_auc_score(y_test, my_pred_test)

    color = 'green' if my_roc_train >= my_roc_train_results[depth - 1] else 'red'
    print(f'{depth}', colored(f'{my_roc_train:18.4f} {my_roc_train_results[depth - 1]:15.4f}', color), end='')
    
    color = 'green' if my_roc_test >= my_roc_test_results[depth - 1] else 'red'
    print(colored(f'{my_roc_test:15.4f} {my_roc_test_results[depth - 1]:15.4f}', color))

depth 	 new roc train 	 old roc train 	 new roc test 	 old roc test
1             0.5000          0.5000         0.5000          0.5000
2             0.7625          0.7625         0.8108          0.8108
3             0.7625          0.7625         0.8108          0.8108
4             0.7949          0.8055         0.8524          0.8524
5             0.7976          0.8020         0.7895          0.7895
6             0.8047          0.8090         0.7810          0.7810
7             0.8319          0.8245         0.7794          0.7641
8             0.8381          0.8279         0.7955          0.7641
9             0.8485          0.8411         0.8032          0.8362
